<a href="https://colab.research.google.com/github/ceyda125/AIDEM-Chatbot/blob/main/IADEMChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Colab hücresine
!pip install -q sentence-transformers faiss-cpu fastapi uvicorn nest-asyncio pyngrok openai


In [ ]:
#en son bunu yaptım!

import re
import json

# --- 1. DOSYAYI OKU ---
file_path = "/content/drive/MyDrive/EnerjiChatbot/enerji_rehberi.txt"
with open(file_path, "r", encoding="utf-8") as f:
    energy_text = f.read()

# --- 2. METNİ TEMİZLEYEN VE PARÇALAYAN FONKSİYON ---
def chunk_text(text, max_words=120):
    text = re.sub(r'\n+', '\n', text).strip()

    # Satırlara böl
    lines = text.split("\n")

    chunks = []
    current_title = ""
    current_content = ""

    for line in lines:
        line = line.strip()
        if not line:
            continue

        if line.startswith("Konu:"):
            # Önceki konu varsa chunk yap
            if current_title:
                full_text = current_title + "\n" + current_content
                words = full_text.split()
                if len(words) <= max_words:
                    chunks.append(full_text.strip())
                else:
                    for i in range(0, len(words), max_words):
                        chunks.append(" ".join(words[i:i+max_words]))
            # Yeni konu başlığı
            current_title = line
            current_content = ""
        else:
            if current_content:
                current_content += " " + line
            else:
                current_content = line

    # Son konuyu ekle
    if current_title:
        full_text = current_title + "\n" + current_content
        words = full_text.split()
        if len(words) <= max_words:
            chunks.append(full_text.strip())
        else:
            for i in range(0, len(words), max_words):
                chunks.append(" ".join(words[i:i+max_words]))

    return chunks

# --- 3. FONKSİYONU ÇALIŞTIR ---
chunks = chunk_text(energy_text, max_words=120)

# --- 4. JSON olarak kaydet ---
json_path = "/content/drive/MyDrive/EnerjiChatbot/enerji_chunks.json"
json_data = [{"id": i, "text": c} for i, c in enumerate(chunks)]

with open(json_path, "w", encoding="utf-8") as f:
    json.dump(json_data, f, ensure_ascii=False, indent=2)

print(f"Toplam chunk sayısı: {len(chunks)}")
print("İlk 5 chunk:")
for c in chunks[:5]:
    print("----")
    print(c)


In [ ]:
# Gerekli kütüphaneler
import json
import numpy as np
from sentence_transformers import SentenceTransformer

# --- JSON'dan chunk'ları oku ---
json_path = "/content/drive/MyDrive/EnerjiChatbot/enerji_chunks.json"

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

texts = [chunk["text"] for chunk in data]
print(f"Toplam chunk sayısı: {len(texts)}")

# --- SentenceTransformer ile embedding oluştur ---
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


# Embeddingleri üret
embeddings = model.encode(texts, show_progress_bar=True)

# FAISS ile uyumlu olacak şekilde float32 tipine çevir
embeddings = np.array(embeddings).astype("float32")

# --- Embeddingleri kaydet ---
embeddings_path = "/content/drive/MyDrive/EnerjiChatbot/embeddings.npy"
np.save(embeddings_path, embeddings)

print(f"Embeddingler kaydedildi: {embeddings_path}")


In [ ]:
import faiss
import numpy as np

# --- Embeddingleri yükle ---
embeddings_path = "/content/drive/MyDrive/EnerjiChatbot/embeddings.npy"
embeddings = np.load(embeddings_path)

# --- FAISS index oluştur ---
embedding_dim = embeddings.shape[1]  # boyut sayısı
index = faiss.IndexFlatL2(embedding_dim)  # CPU üzerinde L2 mesafesi ile

# --- Embeddingleri indexe ekle ---
index.add(embeddings)
print(f"Toplam vektör sayısı indexte: {index.ntotal}")

# --- Indexi kaydet ---
index_path = "/content/drive/MyDrive/EnerjiChatbot/faiss_index.index"
faiss.write_index(index, index_path)

print(f"FAISS index kaydedildi: {index_path}")


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# --- Embedding modelini yükle ---
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


# --- FAISS indexi yükle ---
index_path = "/content/drive/MyDrive/EnerjiChatbot/faiss_index.index"
index = faiss.read_index(index_path)

# --- Chunk verilerini json'dan yükle ---
import json
with open("/content/drive/MyDrive/EnerjiChatbot/enerji_chunks.json", "r", encoding="utf-8") as f:
    data = json.load(f)

def query_faiss(user_question, top_k=3):
    # Sorguyu embedding'e çevir
    query_vec = model.encode([user_question])

    # FAISS ile en yakın chunk'ları bul
    distances, indices = index.search(query_vec, top_k)

    results = []
    for dist, idx in zip(distances[0], indices[0]):
        results.append({
            "chunk_id": idx,
            "distance": float(dist),
            "text": data[idx]["text"]
        })
    return results

soru = "Buzdolabını doğru kullanarak enerji tasarrufu nasıl yapabilirim?"
sonuclar = query_faiss(soru, top_k=3)

for r in sonuclar:
    print(f"[Chunk ID {r['chunk_id']}] (Distance: {r['distance']:.4f})")
    print(r['text'])
    print("-"*50)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "Qwen/Qwen2-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
llm = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,  # CPU için
    device_map="cpu"            # CPU kullan
)


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json

# JSON CHUNK DATASINI YÜKLE
with open("/content/drive/MyDrive/EnerjiChatbot/enerji_chunks.json", "r", encoding="utf-8") as f:
    data = json.load(f)

embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

query = "Buzdolabı enerji tasarrufu için ne yapmalıyım?"

query_vec = embedder.encode([query]).astype("float32")


In [ ]:
!pip install faiss-cpu



In [ ]:
index = faiss.read_index("/content/drive/MyDrive/EnerjiChatbot/faiss_index.index")

k = 3
scores, ids = index.search(query_vec, k)

retrieved_chunks = [data[i]["text"] for i in ids[0]]


In [ ]:
context = "\n\n".join(retrieved_chunks)

prompt = f"""
Aşağıdaki metinlere dayanarak kullanıcı sorusunu cevapla.
Metinler:
{context}

Soru: {query}

Doğal bir dille cevap ver.
"""

inputs = tokenizer(prompt, return_tensors="pt")

output = llm.generate(
    **inputs,
    max_new_tokens=300,
    temperature=0.3
)

print(tokenizer.decode(output[0], skip_special_tokens=True))
